In [1]:
import pandas as pd
import sqlite3

In [2]:
import numpy as np
import gc

In [3]:
as_df = pd.read_csv('[...PATH...]/as-identification/ip-asn-map-full-2020.csv',
                    sep=';', dtype={'ip' : str,
                                    'asn' : str,
                                    'holder' : str,
                                    'cdn_asn' : str})

In [4]:
as_df['cdn_asn'].unique()

array([nan, 'Akamai', 'Fastly', 'Google', 'Amazon', 'Edgecast',
       'Microsoft', 'CDN77', 'Cloudflare', 'Facebook', 'Highwinds',
       'StackPath', 'Yahoo', 'Twitter', 'Level3', 'CDNetworks',
       'LeaseWeb CDN', 'Internap', 'Rackspace', 'Taobao', 'AT&T',
       'Limelight', 'Reflected Networks', 'Incapsula', 'CDNvideo',
       'Yottaa', 'CacheFly', 'Zenedge', 'ChinaCache', 'Medianova',
       'Azion', 'SFR', 'NGENIX', 'Telenor', 'BitGravity', 'Aryaka',
       'Ananke'], dtype=object)

In [ ]:
# try to read everything into memory, monitor if this works

conn = sqlite3.connect('[...PATH...]/httparchive_processed/2020-01/assets.db')
try:    

# full column set
#     assets_df = pd.read_sql('''
#                             select  [index],
#                                     base_page, asset_url,
#                                     protocol, securityDetails AS tls_version,
#                                     asset_content_type, mime_type,
#                                     object_size, 
#                                     is_ad, is_tracker,
#                                     asset_cdn,
#                                     cdn_1, cdn_2, cdn_3,
#                                     ip_addr, cdn_asn,
#                                     hybrid_ident
#                             from    assets_processed
#                             ''',
#                             con=conn)

# reduced column set
    assets_df = pd.read_sql('''
                            select  [index],
                                    base_page, asset_url,
                                    mime_type,
                                    object_size, 
                                    is_ad, is_tracker,
                                    asset_cdn,
                                    cdn_1, cdn_2, cdn_3,
                                    cdn_asn,
                                    hybrid_ident
                            from    assets_processed
                            ''',
                            con=conn)

except Exception as e:
    print(e)
finally:
    conn.close()

In [ ]:
conn = sqlite3.connect('[...PATH...]/httparchive_processed/2020-01/basepages.db')
try:
    bp_1_df = pd.read_sql('''
                          select  base_page, base_page_cdn, score_cdn, base_page_cname
                          from    base_pages
                          ''',
                          con=conn)
    
    bp_2_df = pd.read_sql('''
                          select  base_page,
                                  cdn_asn,
                                  asset_cdn, cdn_1, cdn_2, cdn_3,
                                  hybrid_ident AS bp_cdn
                          from    base_pages_from_assets_processed
                          ''',
                          con=conn).drop_duplicates('base_page')
except Exception as e:
    print(e)
finally:
    conn.close()

In [13]:
assets_df['hybrid_ident'].unique()

array(['Akamai', 'Fastly', None, 'Google', 'Amazon', 'Edgecast',
       'Microsoft', 'CDN77', 'Cloudflare', 'Twitter', 'Facebook',
       'Highwinds', 'StackPath', 'NetDNA', 'jsDelivr', 'Yahoo', 'Cedexis',
       'CDNetworks', 'Netlify', 'LeaseWeb CDN', 'Level3', 'Internap',
       'section.io', 'Rackspace', 'CDNvideo', 'Instart Logic',
       'WordPress', 'KeyCDN', 'Taobao', 'AT&T', 'Limelight',
       'Sucuri Firewall', 'BelugaCDN', 'OVH CDN', 'Reflected Networks',
       'ChinaNetCenter', 'Incapsula', 'BunnyCDN', 'Reapleaf', 'Yottaa',
       'Cachefly', 'Level 3', 'CDN', 'Zenedge', 'CacheFly', 'Medianova',
       'ChinaCache', 'Rocket CDN', 'Yunjiasu', 'Azion', 'OnApp',
       'KINX CDN', 'Universal CDN', 'TRBCDN', 'GoCache', 'Pressable CDN',
       'SFR', 'Sirv CDN', 'CDNsun', 'NGENIX', 'QUIC.cloud',
       'Myra Security CDN', 'Telenor', 'SwiftCDN', 'BitGravity', 'Aryaka',
       'VegaCDN', 'Airee', 'Advanced Hosters CDN', 'Optimal CDN', 'Naver',
       'XLabs Security', 'PageCDN'

In [16]:
bp_1_df['base_page_cdn'].unique()

array(['', 'Akamai', 'Cloudflare', 'Fastly', 'Amazon CloudFront',
       'Netlify', 'Google', 'Cloudflare, Incapsula', 'OVH CDN',
       'Sucuri Firewall', 'Incapsula', 'Yottaa', 'WordPress',
       'Cloudflare, Amazon CloudFront', 'Zenedge', 'CDN77', 'Highwinds',
       'Cloudflare, Google', 'Cloudflare, Fastly', 'CDN', 'section.io',
       'Microsoft Azure', 'Amazon CloudFront, Incapsula', 'CDNetworks',
       'Edgecast', 'OnApp', 'KINX CDN', 'GoCache', 'ChinaNetCenter',
       'Instart Logic', 'Yunjiasu', 'Limelight', 'Facebook', 'Azion',
       'ChinaCache', 'Cloudflare, OVH CDN', 'Myra Security CDN',
       'jsDelivr', 'Cedexis', 'Yahoo', 'NetDNA', 'Aryaka', 'NGENIX',
       'Cloudflare, Sucuri Firewall', 'Level 3', 'Optimal CDN',
       'Amazon CloudFront, Fastly', 'Rackspace', 'BelugaCDN',
       'Sucuri Firewall, Fastly', 'XLabs Security',
       'Sucuri Firewall, Amazon CloudFront', 'StackPath', 'BunnyCDN',
       'KeyCDN', 'BitGravity', 'Cachefly', 'Fly', 'Airee',
       'Hig

In [18]:
bp_2_df['bp_cdn'].unique()

array([None, 'Akamai', 'Cloudflare', 'Fastly', 'Amazon',
       'Amazon CloudFront', 'Netlify', 'Google', 'Microsoft', 'Rackspace',
       'LeaseWeb CDN', 'AT&T', 'OVH CDN', 'Level3', 'Yahoo',
       'Sucuri Firewall', 'Incapsula', 'Yottaa', 'WordPress', 'Zenedge',
       'Highwinds', 'ChinaCache', 'Taobao', 'section.io',
       'Reflected Networks', 'Internap', 'Edgecast', 'CDNetworks', 'CDN',
       'KINX CDN', 'GoCache', 'SFR', 'Microsoft Azure', 'Yunjiasu',
       'Limelight', 'Azion', 'Telenor', 'Myra Security CDN',
       'ChinaNetCenter', 'CDN77', 'Cedexis', 'NetDNA', 'Aryaka', 'NGENIX',
       'Level 3', 'Facebook', 'XLabs Security', 'BelugaCDN', 'CDNvideo',
       'BunnyCDN', 'KeyCDN', 'BitGravity', 'Fly', 'Instart Logic',
       'StackPath', 'Airee', 'QUIC.cloud', 'OnApp', 'Optimal CDN',
       'Twitter', 'BO.LT', 'Rocket CDN', 'Singular CDN', 'Medianova',
       'SwiftCDN', 'Rev Software', 'Bison Grid', 'Ananke',
       'Universal CDN', 'NYI FTW', 'cubeCDN', 'Zycada Networks

In [ ]:
# normalize names for Amazon and Microsoft

cf_ms_names = {'Amazon CloudFront' : 'Amazon',
               'Microsoft Azure' : 'Microsoft'}

for col in ['cdn_1', 'cdn_2', 'cdn_3', 'cdn_asn', 'hybrid_ident']:
    assets_df[col] = assets_df[col].replace(cf_ms_names)

for col in ['cdn_1', 'cdn_2', 'cdn_3', 'cdn_asn', 'bp_cdn']:
    bp_2_df[col] = bp_2_df[col].replace(cf_ms_names)

bp_1_df['base_page_cdn'] = bp_1_df['base_page_cdn'].replace(cf_ms_names)

In [ ]:
base_pages_df = pd.merge(bp_1_df, bp_2_df, on=['base_page'], how='left')

In [11]:
base_pages_df.columns

Index(['base_page', 'base_page_cdn', 'score_cdn', 'base_page_cname', 'cdn_asn',
       'asset_cdn', 'cdn_1', 'cdn_2', 'cdn_3', 'bp_cdn'],
      dtype='object')

In [12]:
assets_df.columns

Index(['index', 'base_page', 'asset_url', 'mime_type', 'object_size', 'is_ad',
       'is_tracker', 'asset_cdn', 'cdn_1', 'cdn_2', 'cdn_3', 'cdn_asn',
       'hybrid_ident'],
      dtype='object')

In [13]:
gc.collect()

0

# ASSET TYPE DISTRIBUTION
# .../   CDN DISTRIBUTION

### Number of Assets per CDN

In [14]:
assets_per_cdn = assets_df['hybrid_ident'].value_counts().to_frame()

In [15]:
assets_per_cdn.rename(columns={'hybrid_ident' : 'num_assets'}, inplace=True)

In [16]:
num_assets_total = len(assets_df)

In [17]:
num_assets_total

392322373

In [18]:
num_assets_cdn = num_assets_total - assets_df['hybrid_ident'].isna().sum()

In [19]:
num_assets_cdn

221918166

In [20]:
assets_per_cdn['share_total'] = assets_per_cdn['num_assets'] / num_assets_total

In [21]:
assets_per_cdn['share_cdn_assets'] = assets_per_cdn['num_assets'] / num_assets_cdn

In [22]:
assets_per_cdn.round(5).head(10)

,num_assets,share_total,share_cdn_assets
Google,76643539,0.19536,0.34537
Amazon,38885034,0.09912,0.17522
Cloudflare,27486460,0.07006,0.12386
Facebook,17714255,0.04515,0.07982
Akamai,15701880,0.04002,0.07076
Fastly,10780916,0.02748,0.04858
WordPress,4139907,0.01055,0.01866
Twitter,3950271,0.01007,0.01780
Microsoft,3813807,0.00972,0.01719
NetDNA,3600997,0.00918,0.01623


In [23]:
mime_type_cdn = assets_df[assets_df['hybrid_ident'].notna()
                         ].groupby('mime_type'
                                  ).size().reset_index(name='num_cdn_assets'

In [24]:
mime_type_cdn.sort_values(by='num_cdn_assets', ascending=False)

,mime_type,num_cdn_assets
4,image,82613713
5,javascript,64223345
7,text,21676628
3,html,19590470
2,font,14245056
6,no type,8539704
0,application,6303607
8,unidentified,3325130
9,video,1135211
1,audio,265302


In [25]:
mime_type_all = assets_df.groupby('mime_type'
                                 ).size().reset_index(name='num_total_assets')

In [26]:
mime_type_all.sort_values(by='num_total_assets', ascending=False)

,mime_type,num_total_assets
4,image,176660130
5,javascript,100195949
7,text,43017071
3,html,28148091
2,font,16569827
6,no type,12402793
0,application,9220762
8,unidentified,4444411
9,video,1236756
1,audio,426583


In [27]:
gc.collect()

28

In [28]:
mime_type_by_provider = assets_df.fillna('').groupby(['hybrid_ident', 'mime_type']
                                                    ).size().reset_index(name='num_assets')

In [29]:
mime_type_by_provider.sort_values(by=['hybrid_ident', 'num_assets'], ascending=False)

,hybrid_ident,mime_type,num_assets
732,section.io,javascript,377097
730,section.io,html,178556
731,section.io,image,14431
728,section.io,application,10633
734,section.io,text,3396
729,section.io,font,838
735,section.io,unidentified,520
733,section.io,no type,24
725,jsDelivr,javascript,248072
726,jsDelivr,text,170751


In [30]:
types = ['javascript', 'html', 'image', 'audio', 'video', 'font', 'application', 'text', 'no type']
mime_type_cdn[~mime_type_cdn['mime_type'].isin(types)].sum()['num_cdn_assets']

3325130

In [31]:
types = ['javascript', 'html', 'image', 'audio', 'video', 'font', 'application', 'text', 'no type']
mime_type_cdn_other = mime_type_cdn[~mime_type_cdn['mime_type'].isin(types)].sum()['num_cdn_assets']
mime_type_all_other = mime_type_all[~mime_type_all['mime_type'].isin(types)].sum()['num_total_assets']
mime_type_by_provider_other = mime_type_by_provider.groupby('hybrid_ident'
                              ).apply(lambda grp: grp[~grp['mime_type'].isin(types)]['num_assets'].sum())

mime_type_by_provider_other = mime_type_by_provider_other[mime_type_by_provider_other != 0]
mime_type_by_provider_other = pd.DataFrame(mime_type_by_provider_other
                                          ).reset_index().rename(columns={0 : 'num_assets'})
mime_type_by_provider_other['mime_type'] = 'other'
mime_type_by_provider_other = mime_type_by_provider_other[['hybrid_ident', 'mime_type', 'num_assets']]

In [32]:
mime_type_cdn_adj = mime_type_cdn[mime_type_cdn['mime_type'].isin(types)
                                 ].append({'mime_type' : 'other',
                                           'num_cdn_assets' : mime_type_cdn_other
                                          }, ignore_index=True)
mime_type_all_adj = mime_type_all[mime_type_all['mime_type'].isin(types)
                                 ].append({'mime_type' : 'other',
                                           'num_total_assets' : mime_type_all_other
                                          }, ignore_index=True)
mime_type_by_provider_adj = pd.concat([mime_type_by_provider[mime_type_by_provider['mime_type'].isin(types)],
                                      mime_type_by_provider_other])

In [33]:
mime_type_cdn_adj

,mime_type,num_cdn_assets
0,application,6303607
1,audio,265302
2,font,14245056
3,html,19590470
4,image,82613713
5,javascript,64223345
6,no type,8539704
7,text,21676628
8,video,1135211
9,other,3325130


In [34]:
mime_type_all_adj

,mime_type,num_total_assets
0,application,9220762
1,audio,426583
2,font,16569827
3,html,28148091
4,image,176660130
5,javascript,100195949
6,no type,12402793
7,text,43017071
8,video,1236756
9,other,4444411


In [35]:
mime_type_by_provider_adj

,hybrid_ident,mime_type,num_assets
0,,application,2917155
1,,audio,161281
2,,font,2324771
3,,html,8557621
4,,image,94046417
5,,javascript,35972604
6,,no type,3863089
7,,text,21340443
9,,video,101545
10,AT&T,application,2606


In [36]:
mime_types = mime_type_cdn_adj.merge(mime_type_all_adj, on='mime_type')
mime_types['cdn_share'] = mime_types['num_cdn_assets'] / mime_types['num_total_assets']

In [37]:
mime_types

,mime_type,num_cdn_assets,num_total_assets,cdn_share
0,application,6303607,9220762,0.683632
1,audio,265302,426583,0.621924
2,font,14245056,16569827,0.859699
3,html,19590470,28148091,0.695979
4,image,82613713,176660130,0.467642
5,javascript,64223345,100195949,0.640977
6,no type,8539704,12402793,0.688531
7,text,21676628,43017071,0.503908
8,video,1135211,1236756,0.917894
9,other,3325130,4444411,0.748160


In [38]:
mime_types.sum()

mime_type           applicationaudiofonthtmlimagejavascriptno type...
num_cdn_assets                                              221918166
num_total_assets                                            392322373
cdn_share                                                     6.82834
dtype: object

In [39]:
mime_types_provider = mime_type_by_provider_adj.merge(mime_type_all_adj, on='mime_type')
mime_types_provider['share'] = mime_types_provider['num_assets'] / mime_types_provider['num_total_assets']

In [40]:
mime_types_provider = mime_types_provider.sort_values(by=['hybrid_ident', 'mime_type']).round(5).reset_index(drop=True)

In [41]:
mime_types_provider

,hybrid_ident,mime_type,num_assets,num_total_assets,share
0,,application,2917155,9220762,0.31637
1,,audio,161281,426583,0.37808
2,,font,2324771,16569827,0.14030
3,,html,8557621,28148091,0.30402
4,,image,94046417,176660130,0.53236
5,,javascript,35972604,100195949,0.35902
6,,no type,3863089,12402793,0.31147
7,,other,1119281,4444411,0.25184
8,,text,21340443,43017071,0.49609
9,,video,101545,1236756,0.08211


In [42]:
gc.collect()

112

# BASE PAGES

# ASSETS BY BASE PAGE

- distribution of asset CDNs, who hosts what/how much?
- how many assets per base page? group by basepage -> count


In [43]:
asset_dist = assets_df[assets_df['hybrid_ident'].notna()
                      ].groupby('base_page'
                               ).size().reset_index(name='cdn_assets')

In [44]:
all_asset_dist = assets_df.groupby('base_page'
                                  ).size().reset_index(name='assets_total')

In [45]:
assets_by_page = all_asset_dist.merge(asset_dist, on='base_page', how='left')

In [46]:
assets_by_page['cdn_assets'] = assets_by_page['cdn_assets'].fillna(0).astype(int)

In [47]:
assets_by_page['cdn_pen'] = assets_by_page['cdn_assets'] / assets_by_page['assets_total']

In [48]:
assets_by_page

,base_page,assets_total,cdn_assets,cdn_pen
0,http://0-1.life/,30,8,0.266667
1,http://0-1.ru/,63,19,0.301587
2,http://0-100jinzai.com/,84,30,0.357143
3,http://0-24suli.hu/,110,37,0.336364
4,http://0-387au.com/,81,41,0.506173
5,http://0-chromosome.hatenablog.jp/,224,192,0.857143
6,http://0-eds.a.ebscohost.com.aupac.lib.athabas...,27,9,0.333333
7,http://0-eds.a.ebscohost.com.library.metu.edu.tr/,10,0,0.000000
8,http://0-eds.b.ebscohost.com.aupac.lib.athabas...,27,9,0.333333
9,http://0-eds.b.ebscohost.com.library.metu.edu.tr/,10,0,0.000000


In [49]:
gc.collect()

49

# DETAILED BASE PAGES (SPLITTING CDNs)

In [186]:
base_pages = base_pages_df.drop_duplicates('base_page').merge(assets_by_page, on='base_page', how='left')

In [187]:
base_pages.isnull().values.any()

True

In [188]:
base_pages[base_pages['assets_total'].isna()]

,base_page,base_page_cdn,score_cdn,base_page_cname,cdn_asn,asset_cdn,cdn_1,cdn_2,cdn_3,bp_cdn,assets_total,cdn_assets,cdn_pen
110562,http://act1.ff.sdo.com/,,-1,ns2.uugame.com,None,,None,None,None,None,NaN,NaN,NaN
122830,http://smart.dekra.com.br/,,-1,marge.ns.cloudflare.com,None,,None,None,None,None,NaN,NaN,NaN
156500,https://diafygi.github.io/,Fastly,100,ns-1339.awsdns-39.org,Fastly,Fastly,Fastly,None,None,Fastly,NaN,NaN,NaN
158706,https://kaluga.korablik.ru/,,-1,ns4-cloud.nic.ru,None,,None,None,None,None,NaN,NaN,NaN
198713,https://krasnodar.piluli.ru/,,-1,ns4-l2.nic.ru,None,,None,None,None,None,NaN,NaN,NaN
215131,https://ekb.piluli.ru/,,-1,ns3-l2.nic.ru,None,,None,None,None,None,NaN,NaN,NaN
225355,https://www.teknosos.com.tr/,,-1,ns2.teknosos.com.tr,None,,None,None,None,None,NaN,NaN,NaN
271473,https://lyubercy.korablik.ru/,,-1,ns3-l2.nic.ru,None,,None,None,None,None,NaN,NaN,NaN
303106,https://saransk.korablik.ru/,,-1,ns8-cloud.nic.ru,None,,None,None,None,None,NaN,NaN,NaN
482681,https://eubtw.belastingdienst.nl/,,-1,ns2.belastingdienst.nl,None,,None,None,None,None,NaN,NaN,NaN


-> 113 pages without assets

In [189]:
base_pages['non_cdn_assets'] = base_pages['assets_total'] - base_pages['cdn_assets']

In [192]:
base_pages

,base_page,base_page_cdn,score_cdn,base_page_cname,cdn_asn,asset_cdn,cdn_1,cdn_2,cdn_3,bp_cdn,assets_total,cdn_assets,cdn_pen,non_cdn_assets
0,https://www.thejamesroom.com/,,95,dns1.p02.nsone.net,None,,None,None,None,None,48.0,45.0,0.937500,3.0
1,https://staszow.geoportal2.pl/,,0,kb.geobid.pl,None,,None,None,None,None,11.0,0.0,0.000000,11.0
2,http://trustypick.com/,,0,ns6168.hostgator.com,None,,None,None,None,None,32.0,2.0,0.062500,30.0
3,https://www.honorstransfercouncil.org/,,98,ns12.wixdns.net,None,,None,None,None,None,100.0,98.0,0.980000,2.0
4,http://www.expedia.com.sg/,Akamai,98,pdns2.ultradns.net,Akamai,,None,None,None,Akamai,246.0,243.0,0.987805,3.0
5,https://ludusmanila.com/,Cloudflare,84,bob.ns.cloudflare.com,Cloudflare,Cloudflare,Cloudflare,None,None,Cloudflare,26.0,26.0,1.000000,0.0
6,http://ge.ssec.wisc.edu/,,0,dns1.ssec.wisc.edu,None,,None,None,None,None,17.0,0.0,0.000000,17.0
7,https://www.stadtwerke-sommerkino.de/,Fastly,100,dns1.kontent.com,Fastly,Fastly,Fastly,None,None,Fastly,11.0,11.0,1.000000,0.0
8,http://esubmission.ema.europa.eu/,,16,ns1eu.bt.net,None,,None,None,None,None,14.0,2.0,0.142857,12.0
9,http://hakusanladies.com/,,15,ns1.dns.ne.jp,None,,None,None,None,None,129.0,38.0,0.294574,91.0


In [193]:
tmp_df = pd.concat([assets_df[['base_page', 'cdn_1']].drop_duplicates().rename(columns={'cdn_1' : 'cdn_name'}),
                   assets_df[['base_page', 'cdn_2']].drop_duplicates().rename(columns={'cdn_2' : 'cdn_name'}),
                   assets_df[['base_page', 'cdn_3']].drop_duplicates().rename(columns={'cdn_3' : 'cdn_name'}),
                   assets_df[['base_page', 'cdn_asn']].drop_duplicates().rename(columns={'cdn_asn' : 'cdn_name'}),
                   base_pages_df[['base_page', 'bp_cdn']].drop_duplicates().rename(columns={'bp_cdn' : 'cdn_name'})  # FIX
                   ]
                  ).sort_values(by=['base_page', 'cdn_name'])

In [194]:
tmp2 = tmp_df.dropna()

In [195]:
cdns_per_page = tmp2.drop_duplicates().groupby('base_page'
                                              ).agg({'cdn_name' : [','.join, 'count']}
                                                   ).rename(columns={'join' : 'cdn_providers',
                                                                     'count' : 'num_cdn'
                                                                    }
                                                           )

In [196]:
cdns_per_page.columns = cdns_per_page.columns.get_level_values(1)
cdns_per_page = cdns_per_page.reset_index()

In [197]:
cdns_per_page.head()

,base_page,cdn_providers,num_cdn
0,http://0-1.life/,"Amazon,Google,Highwinds",3
1,http://0-1.ru/,Google,1
2,http://0-100jinzai.com/,"Facebook,Google",2
3,http://0-24suli.hu/,"Google,Highwinds",2
4,http://0-387au.com/,"Akamai,Cloudflare,Facebook,Google,Highwinds,Ne...",7


In [198]:
base_pages = base_pages.merge(cdns_per_page, on='base_page', how='left')

In [199]:
for col in ['cdn_assets', 'non_cdn_assets', 'assets_total', 'cdn_pen', 'num_cdn']:
    base_pages[col] = base_pages[col].fillna(0) 

In [200]:
base_pages['cdn_providers'] = base_pages['cdn_providers'].fillna('')

In [201]:
base_pages[['cdn_assets', 'non_cdn_assets', 'assets_total', 'cdn_pen', 'num_cdn']].isnull().values.any()

False

In [202]:
base_pages.head()

,base_page,base_page_cdn,score_cdn,base_page_cname,cdn_asn,asset_cdn,cdn_1,cdn_2,cdn_3,bp_cdn,assets_total,cdn_assets,cdn_pen,non_cdn_assets,cdn_providers,num_cdn
0,https://www.thejamesroom.com/,,95,dns1.p02.nsone.net,None,,None,None,None,None,48.0,45.0,0.937500,3.0,"Akamai,Amazon,Fastly,Google",4.0
1,https://staszow.geoportal2.pl/,,0,kb.geobid.pl,None,,None,None,None,None,11.0,0.0,0.000000,11.0,,0.0
2,http://trustypick.com/,,0,ns6168.hostgator.com,None,,None,None,None,None,32.0,2.0,0.062500,30.0,Google,1.0
3,https://www.honorstransfercouncil.org/,,98,ns12.wixdns.net,None,,None,None,None,None,100.0,98.0,0.980000,2.0,"Amazon,Fastly,Google",3.0
4,http://www.expedia.com.sg/,Akamai,98,pdns2.ultradns.net,Akamai,,None,None,None,Akamai,246.0,243.0,0.987805,3.0,"Akamai,Amazon,CDN77,Edgecast,Fastly,Google,Mic...",7.0


In [203]:
# need to split base_page_cdns into separate columns to correctly count distinct number of CDNs
# cdn_providers and num_cdns currently only counts asset CDNs WITHOUT base page information
base_pages['base_page_cdn'].str.count(',').max()

2

In [204]:
bp = base_pages.copy()

In [205]:
tmp = bp['base_page_cdn'].str.split(",", expand=True)  #.replace({'' : None})

bp['bp_cdn_1'] = tmp[0].str.strip()
bp['bp_cdn_2'] = tmp[1].str.strip()

In [206]:
to_replace = {'Amazon CloudFront' : 'Amazon',
              'Microsoft Azure' : 'Microsoft',
              None : ''}

bp['bp_cdn_1'].replace(to_replace, inplace=True)
bp['bp_cdn_2'].replace(to_replace, inplace=True)

In [207]:
bp['bp_cdn_1_counted'] = bp.apply(lambda x: x['bp_cdn_1'] in x['cdn_providers'], axis=1)
bp['bp_cdn_2_counted'] = bp.apply(lambda x: x['bp_cdn_2'] in x['cdn_providers'], axis=1)

In [208]:
bp_backup = bp.copy()

In [209]:
# if base page CDNs are not already considered in the cdn_providers list (determined through asset CDNs), 
# increment counter, add them to list and mark them as counted

bp.loc[~bp['bp_cdn_1_counted'], 'num_cdn'] += 1
bp.loc[~bp['bp_cdn_1_counted'], 'cdn_providers'] = (bp.loc[~bp['bp_cdn_1_counted'], 'cdn_providers'] + ',' + bp.loc[~bp['bp_cdn_1_counted'], 'bp_cdn_1']).str.strip(',')
bp.loc[~bp['bp_cdn_1_counted'], 'bp_cdn_1_counted'] = True

bp.loc[~bp['bp_cdn_2_counted'], 'num_cdn'] += 1
bp.loc[~bp['bp_cdn_2_counted'], 'cdn_providers'] = (bp.loc[~bp['bp_cdn_2_counted'], 'cdn_providers'] + ',' + bp.loc[~bp['bp_cdn_2_counted'], 'bp_cdn_2']).str.strip(',')
bp.loc[~bp['bp_cdn_2_counted'], 'bp_cdn_2_counted'] = True


In [210]:
# verify if it worked
bp.iloc[bp_backup[~bp_backup['bp_cdn_2_counted']].index[0]]

base_page           https://foodscene.deliveroo.hk/
base_page_cdn                    Cloudflare, Fastly
score_cdn                                       100
base_page_cname              mona.ns.cloudflare.com
cdn_asn                                  Cloudflare
asset_cdn                        Cloudflare, Fastly
cdn_1                                    Cloudflare
cdn_2                                        Fastly
cdn_3                                          None
bp_cdn                                   Cloudflare
assets_total                                      1
cdn_assets                                        1
cdn_pen                                           1
non_cdn_assets                                    0
cdn_providers                     Cloudflare,Fastly
num_cdn                                           2
bp_cdn_1                                 Cloudflare
bp_cdn_2                                     Fastly
bp_cdn_1_counted                               True
bp_cdn_2_cou

In [211]:
bp

,base_page,base_page_cdn,score_cdn,base_page_cname,cdn_asn,asset_cdn,cdn_1,cdn_2,cdn_3,bp_cdn,assets_total,cdn_assets,cdn_pen,non_cdn_assets,cdn_providers,num_cdn,bp_cdn_1,bp_cdn_2,bp_cdn_1_counted,bp_cdn_2_counted
0,https://www.thejamesroom.com/,,95,dns1.p02.nsone.net,None,,None,None,None,None,48.0,45.0,0.937500,3.0,"Akamai,Amazon,Fastly,Google",4.0,,,True,True
1,https://staszow.geoportal2.pl/,,0,kb.geobid.pl,None,,None,None,None,None,11.0,0.0,0.000000,11.0,,0.0,,,True,True
2,http://trustypick.com/,,0,ns6168.hostgator.com,None,,None,None,None,None,32.0,2.0,0.062500,30.0,Google,1.0,,,True,True
3,https://www.honorstransfercouncil.org/,,98,ns12.wixdns.net,None,,None,None,None,None,100.0,98.0,0.980000,2.0,"Amazon,Fastly,Google",3.0,,,True,True
4,http://www.expedia.com.sg/,Akamai,98,pdns2.ultradns.net,Akamai,,None,None,None,Akamai,246.0,243.0,0.987805,3.0,"Akamai,Amazon,CDN77,Edgecast,Fastly,Google,Mic...",7.0,Akamai,,True,True
5,https://ludusmanila.com/,Cloudflare,84,bob.ns.cloudflare.com,Cloudflare,Cloudflare,Cloudflare,None,None,Cloudflare,26.0,26.0,1.000000,0.0,"Cloudflare,Edgecast,Google,Twitter",4.0,Cloudflare,,True,True
6,http://ge.ssec.wisc.edu/,,0,dns1.ssec.wisc.edu,None,,None,None,None,None,17.0,0.0,0.000000,17.0,,0.0,,,True,True
7,https://www.stadtwerke-sommerkino.de/,Fastly,100,dns1.kontent.com,Fastly,Fastly,Fastly,None,None,Fastly,11.0,11.0,1.000000,0.0,"Amazon,Fastly",2.0,Fastly,,True,True
8,http://esubmission.ema.europa.eu/,,16,ns1eu.bt.net,None,,None,None,None,None,14.0,2.0,0.142857,12.0,Google,1.0,,,True,True
9,http://hakusanladies.com/,,15,ns1.dns.ne.jp,None,,None,None,None,None,129.0,38.0,0.294574,91.0,Google,1.0,,,True,True


In [212]:
bp.dtypes

base_page            object
base_page_cdn        object
score_cdn            object
base_page_cname      object
cdn_asn              object
asset_cdn            object
cdn_1                object
cdn_2                object
cdn_3                object
bp_cdn               object
assets_total        float64
cdn_assets          float64
cdn_pen             float64
non_cdn_assets      float64
cdn_providers        object
num_cdn             float64
bp_cdn_1             object
bp_cdn_2             object
bp_cdn_1_counted       bool
bp_cdn_2_counted       bool
dtype: object

In [213]:
base_pages = bp[['base_page', 'base_page_cname',
                 'base_page_cdn', 'bp_cdn_1', 'bp_cdn_2', 'cdn_asn', 'bp_cdn',
                 'num_cdn', 'cdn_providers',
                 'cdn_assets', 'non_cdn_assets', 'assets_total', 'cdn_pen',
                 'score_cdn',
                ]]

In [ ]:
for col in ['score_cdn']:
    base_pages[col] = base_pages[col].astype(float)
    # CDN Score is calculated by WebPagetest for a base page;
    # similar to penetration metric but slightly different by definition
    # i.e., considers static resources (score) vs all resources (penetration)

    # some analysis below but not described in paper to avoid confusion

In [ ]:
base_pages['score_cdn'] = base_pages['score_cdn'] / 100.0

In [216]:
base_pages.dtypes

base_page           object
base_page_cname     object
base_page_cdn       object
bp_cdn_1            object
bp_cdn_2            object
cdn_asn             object
bp_cdn              object
num_cdn            float64
cdn_providers       object
cdn_assets         float64
non_cdn_assets     float64
assets_total       float64
cdn_pen            float64
score_cdn          float64
dtype: object

In [217]:
base_pages

,base_page,base_page_cname,base_page_cdn,bp_cdn_1,bp_cdn_2,cdn_asn,bp_cdn,num_cdn,cdn_providers,cdn_assets,non_cdn_assets,assets_total,cdn_pen,score_cdn
0,https://www.thejamesroom.com/,dns1.p02.nsone.net,,,,None,None,4.0,"Akamai,Amazon,Fastly,Google",45.0,3.0,48.0,0.937500,0.95
1,https://staszow.geoportal2.pl/,kb.geobid.pl,,,,None,None,0.0,,0.0,11.0,11.0,0.000000,0.00
2,http://trustypick.com/,ns6168.hostgator.com,,,,None,None,1.0,Google,2.0,30.0,32.0,0.062500,0.00
3,https://www.honorstransfercouncil.org/,ns12.wixdns.net,,,,None,None,3.0,"Amazon,Fastly,Google",98.0,2.0,100.0,0.980000,0.98
4,http://www.expedia.com.sg/,pdns2.ultradns.net,Akamai,Akamai,,Akamai,Akamai,7.0,"Akamai,Amazon,CDN77,Edgecast,Fastly,Google,Mic...",243.0,3.0,246.0,0.987805,0.98
5,https://ludusmanila.com/,bob.ns.cloudflare.com,Cloudflare,Cloudflare,,Cloudflare,Cloudflare,4.0,"Cloudflare,Edgecast,Google,Twitter",26.0,0.0,26.0,1.000000,0.84
6,http://ge.ssec.wisc.edu/,dns1.ssec.wisc.edu,,,,None,None,0.0,,0.0,17.0,17.0,0.000000,0.00
7,https://www.stadtwerke-sommerkino.de/,dns1.kontent.com,Fastly,Fastly,,Fastly,Fastly,2.0,"Amazon,Fastly",11.0,0.0,11.0,1.000000,1.00
8,http://esubmission.ema.europa.eu/,ns1eu.bt.net,,,,None,None,1.0,Google,2.0,12.0,14.0,0.142857,0.16
9,http://hakusanladies.com/,ns1.dns.ne.jp,,,,None,None,1.0,Google,38.0,91.0,129.0,0.294574,0.15


In [218]:
base_pages[base_pages['bp_cdn_2'] != '']

,base_page,base_page_cname,base_page_cdn,bp_cdn_1,bp_cdn_2,cdn_asn,bp_cdn,num_cdn,cdn_providers,cdn_assets,non_cdn_assets,assets_total,cdn_pen,score_cdn
102,http://www.palacrunowo.pl/,ns2.dyalcom.com.pl,"Cloudflare, Incapsula",Cloudflare,Incapsula,None,Cloudflare,3.0,"Cloudflare,Google,Incapsula",36.0,11.0,47.0,0.765957,0.78
711,https://www.saratogaresortvillas.com/,pdns06.domaincontrol.com,"Cloudflare, Amazon CloudFront",Cloudflare,Amazon,Cloudflare,Cloudflare,5.0,"Amazon,Cedexis,Cloudflare,Facebook,Google",47.0,7.0,54.0,0.870370,0.83
900,https://www.decomeubel.nl/,ns1.hostnet.nl,"Cloudflare, Google",Cloudflare,Google,Cloudflare,Cloudflare,4.0,"Amazon,Cloudflare,Google,Highwinds",185.0,3.0,188.0,0.984043,0.98
952,https://next.preciousplastic.com/,drew.ns.cloudflare.com,"Cloudflare, Fastly",Cloudflare,Fastly,Cloudflare,Cloudflare,4.0,"Cloudflare,Fastly,Google,Highwinds",144.0,0.0,144.0,1.000000,0.98
1034,https://www.monmouthkitchen.co.uk/,ns2.domainmonster.com,"Cloudflare, Amazon CloudFront",Cloudflare,Amazon,Amazon,Amazon,5.0,"Akamai,Amazon,Cloudflare,Google,Microsoft",54.0,3.0,57.0,0.947368,0.81
1072,https://salesflare.com/,matt.ns.cloudflare.com,"Cloudflare, Fastly",Cloudflare,Fastly,Cloudflare,Cloudflare,8.0,"Akamai,Amazon,Cloudflare,Facebook,Fastly,Googl...",90.0,2.0,92.0,0.978261,0.92
1110,https://www.123paracord.nl/,ns0.transip.net,"Cloudflare, Google",Cloudflare,Google,Cloudflare,Cloudflare,5.0,"Cloudflare,Facebook,Google,Highwinds,Microsoft",94.0,5.0,99.0,0.949495,0.91
1526,https://www.megajatek.hu/,bruce.ns.cloudflare.com,"Cloudflare, Google",Cloudflare,Google,Cloudflare,Cloudflare,4.0,"Akamai,Cloudflare,Google,Microsoft",47.0,0.0,47.0,1.000000,1.00
1762,https://www.towngasappliance.com/,tkons01.towngastelecom.com,"Amazon CloudFront, Incapsula",Amazon,Incapsula,Incapsula,Amazon,4.0,"Amazon,Google,Highwinds,Incapsula",45.0,0.0,45.0,1.000000,1.00
1978,https://thekdaily.com/,ed.ns.cloudflare.com,"Cloudflare, Google",Cloudflare,Google,Cloudflare,Cloudflare,8.0,"Akamai,Amazon,CDN77,Cloudflare,Facebook,Fastly...",249.0,19.0,268.0,0.929104,0.97


In [219]:
base_pages_cdn = base_pages[(base_pages['base_page_cdn'] != '') | (base_pages['bp_cdn'].notna())]

In [220]:
base_pages_no_cdn = base_pages[(base_pages['base_page_cdn'] == '') & (base_pages['bp_cdn'].isna())]

In [221]:
len(base_pages_cdn)

1375025

In [222]:
len(base_pages_no_cdn)

2903601

In [223]:
len(base_pages)

4278626

In [224]:
len(base_pages_cdn) + len(base_pages_no_cdn)

4278626

In [225]:
len(base_pages_cdn) / len(base_pages)

0.32137069236712906

In [226]:
base_pages_cdn.head()

,base_page,base_page_cname,base_page_cdn,bp_cdn_1,bp_cdn_2,cdn_asn,bp_cdn,num_cdn,cdn_providers,cdn_assets,non_cdn_assets,assets_total,cdn_pen,score_cdn
4,http://www.expedia.com.sg/,pdns2.ultradns.net,Akamai,Akamai,,Akamai,Akamai,7.0,"Akamai,Amazon,CDN77,Edgecast,Fastly,Google,Mic...",243.0,3.0,246.0,0.987805,0.98
5,https://ludusmanila.com/,bob.ns.cloudflare.com,Cloudflare,Cloudflare,,Cloudflare,Cloudflare,4.0,"Cloudflare,Edgecast,Google,Twitter",26.0,0.0,26.0,1.000000,0.84
7,https://www.stadtwerke-sommerkino.de/,dns1.kontent.com,Fastly,Fastly,,Fastly,Fastly,2.0,"Amazon,Fastly",11.0,0.0,11.0,1.000000,1.00
13,https://editmp3.online/,cheryl.ns.cloudflare.com,Cloudflare,Cloudflare,,Cloudflare,Cloudflare,6.0,"Akamai,Amazon,Cloudflare,Google,Highwinds,Stac...",118.0,6.0,124.0,0.951613,0.94
14,https://www.toyotacertifiedcenterofsantee.com/,ns1.dotster.com,,,,Amazon,Amazon,7.0,"Akamai,Amazon,Edgecast,Facebook,Google,Highwin...",50.0,1.0,51.0,0.980392,0.50


In [227]:
base_pages_cdn[base_pages_cdn['num_cdn'] == 0]

,base_page,base_page_cname,base_page_cdn,bp_cdn_1,bp_cdn_2,cdn_asn,bp_cdn,num_cdn,cdn_providers,cdn_assets,non_cdn_assets,assets_total,cdn_pen,score_cdn


In [230]:
assert len(base_pages_cdn[base_pages_cdn['num_cdn'] == 0]) == 0

In [228]:
base_pages_no_cdn.head()

,base_page,base_page_cname,base_page_cdn,bp_cdn_1,bp_cdn_2,cdn_asn,bp_cdn,num_cdn,cdn_providers,cdn_assets,non_cdn_assets,assets_total,cdn_pen,score_cdn
0,https://www.thejamesroom.com/,dns1.p02.nsone.net,,,,None,None,4.0,"Akamai,Amazon,Fastly,Google",45.0,3.0,48.0,0.9375,0.95
1,https://staszow.geoportal2.pl/,kb.geobid.pl,,,,None,None,0.0,,0.0,11.0,11.0,0.0000,0.00
2,http://trustypick.com/,ns6168.hostgator.com,,,,None,None,1.0,Google,2.0,30.0,32.0,0.0625,0.00
3,https://www.honorstransfercouncil.org/,ns12.wixdns.net,,,,None,None,3.0,"Amazon,Fastly,Google",98.0,2.0,100.0,0.9800,0.98
6,http://ge.ssec.wisc.edu/,dns1.ssec.wisc.edu,,,,None,None,0.0,,0.0,17.0,17.0,0.0000,0.00


In [229]:
base_pages_no_cdn[base_pages_no_cdn['num_cdn'] == 0]

,base_page,base_page_cname,base_page_cdn,bp_cdn_1,bp_cdn_2,cdn_asn,bp_cdn,num_cdn,cdn_providers,cdn_assets,non_cdn_assets,assets_total,cdn_pen,score_cdn
1,https://staszow.geoportal2.pl/,kb.geobid.pl,,,,None,None,0.0,,0.0,11.0,11.0,0.0,0.00
6,http://ge.ssec.wisc.edu/,dns1.ssec.wisc.edu,,,,None,None,0.0,,0.0,17.0,17.0,0.0,0.00
11,http://www.motosportklub.cz/,ns2.server4you.cz,,,,None,None,0.0,,0.0,16.0,16.0,0.0,0.00
44,https://universiteiten.rechtsorde.nl/,ns2.observatoiredesmarques.fr,,,,None,None,0.0,,0.0,27.0,27.0,0.0,0.00
49,http://www.sunrain.jp/,ns5.network.jp,,,,None,None,0.0,,0.0,14.0,14.0,0.0,0.00
75,http://notariafelixjaracadot.cl/,dns3.hostingplus.cl,,,,None,None,0.0,,0.0,17.0,17.0,0.0,0.00
77,http://www.takesend.com/,dns22.hichina.com,,,,None,None,0.0,,0.0,50.0,50.0,0.0,0.00
122,http://www.kjbus.com/,ns2.cafe24.com,,,,None,None,0.0,,0.0,38.0,38.0,0.0,0.00
125,https://wingmachine.webordering.ca/,ns52.domaincontrol.com,,,,None,None,0.0,,0.0,27.0,27.0,0.0,0.00
134,http://www.procura.cagliari.giustizia.it/,ns1a.btitalia.it,,,,None,None,0.0,,0.0,54.0,54.0,0.0,0.00


In [105]:
import Cdf
import Pmf

In [106]:
def get_cdf(df, col):
    df_tmp = df[df[col] >= 0][col].dropna()
    print(len(df_tmp))
    return Cdf.MakeCdfFromList(df_tmp)

In [231]:
page_scores_cdn_cdf = get_cdf(base_pages_cdn, 'score_cdn')
page_scores_no_cdn_cdf = get_cdf(base_pages_no_cdn, 'score_cdn')

1372961
2882484


In [232]:
page_cdn_pen_cdn_cdf = get_cdf(base_pages_cdn, 'cdn_pen')
page_cdn_pen_no_cdn_cdf = get_cdf(base_pages_no_cdn, 'cdn_pen')

1375025
2903601


In [233]:
page_cdn_assets_cdn_cdf = get_cdf(base_pages_cdn, 'cdn_assets')
page_cdn_assets_no_cdn_cdf = get_cdf(base_pages_no_cdn, 'cdn_assets')

1375025
2903601


In [234]:
page_non_cdn_assets_cdn_cdf = get_cdf(base_pages_cdn, 'non_cdn_assets')
page_non_cdn_assets_no_cdn_cdf = get_cdf(base_pages_no_cdn, 'non_cdn_assets')

1375025
2903601


In [235]:
page_assets_total_cdn_cdf = get_cdf(base_pages_cdn, 'assets_total')
page_assets_total_no_cdn_cdf = get_cdf(base_pages_no_cdn, 'assets_total')

1375025
2903601


In [236]:
# all pages, not split by CDN/no CDN
cdn_assets_by_page_cdf = get_cdf(base_pages, 'cdn_assets')
non_cdn_assets_by_page_cdf = get_cdf(base_pages, 'non_cdn_assets')
total_assets_by_page_cdf = get_cdf(base_pages, 'assets_total')
cdn_pen_by_page = get_cdf(base_pages, 'cdn_pen')


4278626
4278626
4278626
4278626


### CDN Pen based on Base Pages only (for comparison with other measurements)

In [237]:
len(base_pages_cdn['base_page'].unique())

1375025

In [238]:
len(base_pages_no_cdn['base_page'].unique())

2903601

In [254]:
len(base_pages_cdn['base_page'].unique()) / len(base_pages['base_page'].unique())

0.32137069236712906

### <span style="color:blue"> => around 32.1% Base Page CDN Pen based on HTTP Archive Dataset </span>

# Saving Results

In [116]:
import dill

In [117]:
class DillSerializer:
   
    def __init__(self, path):
        self.path = path
   
    def serialize(self, obj, protocol_version=2):
        f = open(self.path, "wb+")
        dill.dump(obj, f, protocol_version)
        f.close()
       
    def deserialize(self):
        f = open(self.path, "rb")
        data = dill.load(f)
        f.close()
        return data

In [240]:
per_page_cdfs = [(cdn_assets_by_page_cdf, 'num_cdn_assets_per_page'),
                 (non_cdn_assets_by_page_cdf, 'num_non_cdn_assets_per_page'),
                 (total_assets_by_page_cdf, 'num_total_assets_per_page'),
                 (cdn_pen_by_page, 'cdn_pen_per_page'),
                 
#                  (cdn_pen_by_cdn_page, 'cdn_pen_per_cdn_page'),
#                  (cdn_pen_by_non_cdn_page, 'cdn_pen_per_non_cdn_page')
                 
                ]

for cdf_obj, cdf_name in per_page_cdfs:
    DillSerializer(path='dills/per-page-cdfs/%s.dill' % cdf_name).serialize(cdf_obj)

In [241]:
base_page_cdfs = [(page_scores_cdn_cdf, 'scores_cdn'),
                  (page_scores_no_cdn_cdf, 'scores_no_cdn'),

                  (page_cdn_assets_cdn_cdf, 'cdn_assets_cdn'),
                  (page_cdn_assets_no_cdn_cdf, 'cdn_assets_no_cdn'),
                
                  (page_non_cdn_assets_cdn_cdf, 'non_cdn_assets_cdn'),
                  (page_non_cdn_assets_no_cdn_cdf, 'non_cdn_assets_no_cdn'),
                  
                  (page_assets_total_cdn_cdf, 'assets_total_cdn'),
                  (page_assets_total_no_cdn_cdf, 'assets_total_no_cdn'),
                
                  (page_cdn_pen_cdn_cdf, 'cdn_pen_cdn'),
                  (page_cdn_pen_no_cdn_cdf, 'cdn_pen_no_cdn'),
                 ]

for cdf_obj, cdf_name in base_page_cdfs:
    DillSerializer(path='dills/base-pages/%s.dill' % cdf_name).serialize(cdf_obj)

In [242]:
num_cdn_bp_with_cdn_cdf = Cdf.MakeCdfFromList(base_pages_cdn['num_cdn'])
num_cdn_bp_without_cdn_cdf = Cdf.MakeCdfFromList(base_pages_no_cdn['num_cdn'])

In [243]:
DillSerializer(path='dills/num_cdn_bp_with_cdn_cdf.dill').serialize(num_cdn_bp_with_cdn_cdf)
DillSerializer(path='dills/num_cdn_bp_without_cdn_cdf.dill').serialize(num_cdn_bp_without_cdn_cdf)

In [244]:
num_cdn_cdf = Cdf.MakeCdfFromList(base_pages['num_cdn'])

In [245]:
DillSerializer(path='dills/num_cdn_cdf.dill').serialize(num_cdn_cdf)

In [125]:
import gc
gc.collect()

64

In [246]:
base_pages.head()

,base_page,base_page_cname,base_page_cdn,bp_cdn_1,bp_cdn_2,cdn_asn,bp_cdn,num_cdn,cdn_providers,cdn_assets,non_cdn_assets,assets_total,cdn_pen,score_cdn
0,https://www.thejamesroom.com/,dns1.p02.nsone.net,,,,None,None,4.0,"Akamai,Amazon,Fastly,Google",45.0,3.0,48.0,0.937500,0.95
1,https://staszow.geoportal2.pl/,kb.geobid.pl,,,,None,None,0.0,,0.0,11.0,11.0,0.000000,0.00
2,http://trustypick.com/,ns6168.hostgator.com,,,,None,None,1.0,Google,2.0,30.0,32.0,0.062500,0.00
3,https://www.honorstransfercouncil.org/,ns12.wixdns.net,,,,None,None,3.0,"Amazon,Fastly,Google",98.0,2.0,100.0,0.980000,0.98
4,http://www.expedia.com.sg/,pdns2.ultradns.net,Akamai,Akamai,,Akamai,Akamai,7.0,"Akamai,Amazon,CDN77,Edgecast,Fastly,Google,Mic...",243.0,3.0,246.0,0.987805,0.98


In [247]:
cdns_per_page.head()

,base_page,cdn_providers,num_cdn
0,http://0-1.life/,"Amazon,Google,Highwinds",3
1,http://0-1.ru/,Google,1
2,http://0-100jinzai.com/,"Facebook,Google",2
3,http://0-24suli.hu/,"Google,Highwinds",2
4,http://0-387au.com/,"Akamai,Cloudflare,Facebook,Google,Highwinds,Ne...",7


In [248]:
mime_types

,mime_type,num_cdn_assets,num_total_assets,cdn_share
0,application,6303607,9220762,0.683632
1,audio,265302,426583,0.621924
2,font,14245056,16569827,0.859699
3,html,19590470,28148091,0.695979
4,image,82613713,176660130,0.467642
5,javascript,64223345,100195949,0.640977
6,no type,8539704,12402793,0.688531
7,text,21676628,43017071,0.503908
8,video,1135211,1236756,0.917894
9,other,3325130,4444411,0.748160


In [249]:
mime_types_provider.head()

,hybrid_ident,mime_type,num_assets,num_total_assets,share
0,,application,2917155,9220762,0.31637
1,,audio,161281,426583,0.37808
2,,font,2324771,16569827,0.14030
3,,html,8557621,28148091,0.30402
4,,image,94046417,176660130,0.53236


In [250]:
assets_per_cdn.head()

,num_assets,share_total,share_cdn_assets
Google,76643539,0.195359,0.345368
Amazon,38885034,0.099115,0.175222
Cloudflare,27486460,0.070061,0.123859
Facebook,17714255,0.045152,0.079823
Akamai,15701880,0.040023,0.070755


In [251]:
assets_per_cdn

,num_assets,share_total,share_cdn_assets
Google,76643539,1.953586e-01,3.453685e-01
Amazon,38885034,9.911501e-02,1.752224e-01
Cloudflare,27486460,7.006090e-02,1.238585e-01
Facebook,17714255,4.515229e-02,7.982337e-02
Akamai,15701880,4.002290e-02,7.075527e-02
Fastly,10780916,2.747974e-02,4.858059e-02
WordPress,4139907,1.055231e-02,1.865511e-02
Twitter,3950271,1.006894e-02,1.780058e-02
Microsoft,3813807,9.721105e-03,1.718565e-02
NetDNA,3600997,9.178668e-03,1.622669e-02


In [252]:
assets_per_cdn.round(5).reset_index()

,index,num_assets,share_total,share_cdn_assets
0,Google,76643539,0.19536,0.34537
1,Amazon,38885034,0.09912,0.17522
2,Cloudflare,27486460,0.07006,0.12386
3,Facebook,17714255,0.04515,0.07982
4,Akamai,15701880,0.04002,0.07076
5,Fastly,10780916,0.02748,0.04858
6,WordPress,4139907,0.01055,0.01866
7,Twitter,3950271,0.01007,0.01780
8,Microsoft,3813807,0.00972,0.01719
9,NetDNA,3600997,0.00918,0.01623


#### Object Sizes

In [263]:
assets_df['object_size_parsed'] = pd.to_numeric(assets_df['object_size'], errors='coerce')

In [266]:
obj_size_per_cdn = assets_df[['hybrid_ident', 'object_size']
                            ].replace({'' : '---'}
                                     ).groupby('hybrid_ident')['object_size_parsed'].sum().reset_index()

In [271]:
obj_size_per_cdn = obj_size_per_cdn.sort_values('object_size_parsed', ascending=False
                                               ).rename(columns={'object_size_parsed' : 'bytes'})

In [274]:
obj_size_per_cdn

,hybrid_ident,bytes
29,Google,1.605106e+12
4,Amazon,1.371403e+12
23,Cloudflare,1.026878e+12
3,Akamai,5.333766e+11
25,Facebook,4.545967e+11
26,Fastly,4.415775e+11
43,Microsoft,1.943458e+11
48,NetDNA,1.594049e+11
24,Edgecast,1.311514e+11
76,WordPress,1.174001e+11


In [253]:
con_out = sqlite3.connect('[...PATH...]/httparchive_processed/2020-01/agg-summaries.db') 
con_out.text_factory = str

base_pages.to_sql('base_pages_detailed', con=con_out, index=False, if_exists='replace')
cdns_per_page.to_sql('cdns_per_page', con=con_out, index=False, if_exists='replace')
obj_size_per_cdn.to_sql('obj_size_per_cdn', con=con_out, index=False, if_exists='replace')
mime_types.to_sql('mime_types', con=con_out, index=False, if_exists='replace')
mime_types_provider.to_sql('mime_types_provider', con=con_out, index=False, if_exists='replace')
assets_per_cdn.round(5).reset_index().rename(columns={'index':'cdn'}
                                            ).to_sql('cdn_provider_distribution',
                                                     con=con_out,
                                                     index=False,
                                                     if_exists='replace')

con_out.close()

# Looking into JavaScript

In [134]:
js_assets = assets_df[assets_df['mime_type'] == 'javascript']

In [135]:
js_assets

,index,base_page,asset_url,mime_type,object_size,is_ad,is_tracker,asset_cdn,cdn_1,cdn_2,cdn_3,cdn_asn,hybrid_ident
0,0,https://www.thejamesroom.com/,https://use.typekit.net/ik/j6yHdzj79DugkCmKnZA...,javascript,7586,0,0,Akamai,Akamai,None,None,Akamai,Akamai
1,1,https://www.thejamesroom.com/,https://assets.squarespace.com/universal/scrip...,javascript,424418,0,0,Fastly,Fastly,None,None,Fastly,Fastly
2,2,https://www.thejamesroom.com/,https://assets.squarespace.com/universal/scrip...,javascript,14928,0,0,Fastly,Fastly,None,None,Fastly,Fastly
6,6,https://www.thejamesroom.com/,https://www.sevenrooms.com/widget/embed.js,javascript,7081,0,0,Google,Google,None,None,Google,Google
7,7,https://www.thejamesroom.com/,https://static1.squarespace.com/static/ta/56d9...,javascript,42303,0,0,Fastly,Fastly,None,None,None,Fastly
8,8,https://www.thejamesroom.com/,https://cf.chownowcdn.com/latest/static/integr...,javascript,4117,0,0,Amazon CloudFront,Amazon,None,None,Amazon,Amazon
71,71,http://trustypick.com/,http://trustypick.com/wp-includes/js/jquery/jq...,javascript,42766,0,0,,None,None,None,None,None
72,72,http://trustypick.com/,http://trustypick.com/wp-includes/js/jquery/jq...,javascript,4444,0,0,,None,None,None,None,None
73,73,http://trustypick.com/,http://trustypick.com/wp-content/plugins/insta...,javascript,1448,0,0,,None,None,None,None,None
74,74,http://trustypick.com/,http://trustypick.com/wp-content/plugins/insta...,javascript,14401,0,0,,None,None,None,None,None


In [136]:
jquery = js_assets[js_assets['asset_url'].str.contains('jquery')]

In [137]:
len(jquery)

14532039

In [138]:
len(jquery)/len(js_assets)

0.14503619303011941

-> jQuery makes up around 14.5% of all JavaScript Assets

In [140]:
jquery_self_hosted = jquery[jquery['hybrid_ident'].isna()]

In [141]:
len(jquery_self_hosted)

9103934

In [142]:
jquery_cdn_hosted = jquery[jquery['hybrid_ident'].notna()]

In [143]:
len(jquery_cdn_hosted)

5428105

In [144]:
len(jquery_cdn_hosted) / len(jquery)

0.37352672945620363

-> around 37.4% of jQuery hosted on CDNs

In [145]:
# number of jQuery assets per provider
jquery_by_provider = jquery['hybrid_ident'].value_counts()

In [146]:
jquery_by_provider.sum()

5428105

In [147]:
# distinct pages that use CDN-hosted jQuery from any source
len(jquery_cdn_hosted['base_page'].unique())

1730883

In [148]:
# base pages that use multiple jQuery scripts from multiple CDNs
mult_jquery_pages = jquery_cdn_hosted.groupby('base_page').filter(lambda x: len(x) > 1)

In [149]:
# number of distinct pages that use multiple jQuery scripts from multiple CDNs
mult_jquery_pages['base_page'].nunique()

965710

In [150]:
jquery_cdn_hosted

,index,base_page,asset_url,mime_type,object_size,is_ad,is_tracker,asset_cdn,cdn_1,cdn_2,cdn_3,cdn_asn,hybrid_ident
193,193,http://www.expedia.com.sg/,https://c.travel-assets.com/uitoolkit/2-229/b7...,javascript,63333,0,0,Akamai,Akamai,None,None,Akamai,Akamai
446,446,https://ludusmanila.com/,https://cdnjs.cloudflare.com/ajax/libs/jquery/...,javascript,32406,0,0,Cloudflare,Cloudflare,None,None,Cloudflare,Cloudflare
794,794,https://editmp3.online/,https://cdnjs.cloudflare.com/ajax/libs/jquery/...,javascript,32920,0,0,Cloudflare,Cloudflare,None,None,Cloudflare,Cloudflare
795,795,https://editmp3.online/,https://cdnjs.cloudflare.com/ajax/libs/jquery....,javascript,6212,0,0,Cloudflare,Cloudflare,None,None,Cloudflare,Cloudflare
1495,1495,https://www.flamingotheaterbar.com/,https://s3.amazonaws.com/b360/flamingo/slider/...,javascript,84249,0,0,,None,None,None,Amazon,Amazon
1662,1662,http://blog.segurbaby.com/,http://ajax.googleapis.com/ajax/libs/jquery/1....,javascript,33593,0,0,Google,Google,None,None,Google,Google
1853,1853,https://secure.rosettastone.fr/,https://code.jquery.com/jquery-3.3.1.min.js,javascript,30288,0,0,Highwinds,Highwinds,None,None,Highwinds,Highwinds
1860,1860,http://bbq-recipe.jp/,http://ajax.googleapis.com/ajax/libs/jquery/1....,javascript,33593,0,0,Google,Google,None,None,Google,Google
1861,1861,http://bbq-recipe.jp/,http://ajax.googleapis.com/ajax/libs/jqueryui/...,javascript,62563,0,0,Google,Google,None,None,Google,Google
1899,1899,http://www.clubinsomniagroup.com/,https://www.clubinsomniagroup.com/wp-includes/...,javascript,3955,0,0,Cloudflare,Cloudflare,None,None,Cloudflare,Cloudflare


In [151]:
# per provider: number of distinct pages 
base_pages_jquery_cdn = jquery_cdn_hosted.groupby('hybrid_ident')['base_page'].nunique()

In [152]:
base_pages_jquery_cdn.sum()

2170461

In [153]:
base_pages_jquery_cdn.sort_values(ascending=False)

hybrid_ident
Google             769777
Cloudflare         427776
Highwinds          255349
Amazon             238365
Akamai              92639
Fastly              68312
Microsoft           61872
WordPress           55213
jsDelivr            33590
NetDNA              24033
Edgecast            19321
Sucuri Firewall     18819
CDNetworks          18650
Rackspace           15461
Incapsula           11360
LeaseWeb CDN         9440
Cachefly             5257
KeyCDN               4977
Taobao               3963
CDNvideo             3408
OVH CDN              3268
CDN                  2842
CDN77                2554
AT&T                 2438
StackPath            2409
Level3               2240
Yahoo                1762
Azion                1740
GoCache              1477
Limelight            1391
                    ...  
BelugaCDN             121
Medianova              88
CDNsun                 71
TRBCDN                 68
BitGravity             60
Singular CDN           55
KINX CDN               44

# ==========================================================

Other misc. JavaScript frameworks

In [154]:
react = js_assets[js_assets['asset_url'].str.contains('react')]

In [155]:
len(react)

270682

In [156]:
angular = js_assets[js_assets['asset_url'].str.contains('angular')]

In [157]:
len(angular)

230915

In [158]:
bootstrap = js_assets[js_assets['asset_url'].str.contains('bootstrap')]

In [159]:
len(bootstrap)

1522324

# Ads / Trackers

In [160]:
ads_df = assets_df[assets_df['is_ad'] == 1]
trackers_df = assets_df[assets_df['is_tracker'] == 1]

In [161]:
len(ads_df)

13175543

In [162]:
len(ads_df) / len(assets_df)

0.03358346071178561

In [163]:
len(trackers_df)

28949586

In [164]:
len(trackers_df) / len(assets_df)

0.07379030101859625

In [165]:
ad_trackers_df = assets_df[(assets_df['is_ad'] == 1) & (assets_df['is_tracker'] == 1)]

In [166]:
len(ad_trackers_df)

258257

In [167]:
len(trackers_df) - len(ad_trackers_df)

28691329

In [168]:
len(ads_df) - len(ad_trackers_df)

12917286

=> around 30M ads, 50M trackers, 10M both (means 20M only-ads, 40M only trackers)

In [169]:
ads_by_type = ads_df['mime_type'].value_counts()

In [170]:
pd.DataFrame({'type_count' : ads_by_type, 'share' : ads_by_type/len(ads_df)}).round(5)

,type_count,share
javascript,4026581,0.30561
html,3726047,0.28280
image,3679336,0.27925
text,624737,0.04742
no type,511782,0.03884
application,410229,0.03114
unidentified,193076,0.01465
font,2734,0.00021
video,996,0.00008
audio,25,0.00000


In [171]:
ads_by_provider = ads_df['hybrid_ident'].fillna('- no CDN -').value_counts()

In [172]:
pd.DataFrame({'prov_count' : ads_by_provider, 'share' : ads_by_provider/len(ads_df)}).round(5)

,prov_count,share
Google,8776465,0.66612
- no CDN -,2715437,0.20610
Amazon,401946,0.03051
Akamai,362619,0.02752
Yahoo,291181,0.02210
Cloudflare,220693,0.01675
Edgecast,123498,0.00937
Fastly,116593,0.00885
Highwinds,32702,0.00248
Internap,21971,0.00167


In [173]:
ads_df.head(10000)[ads_df.head(10000)['hybrid_ident'] == 'Twitter']

,index,base_page,asset_url,mime_type,object_size,is_ad,is_tracker,asset_cdn,cdn_1,cdn_2,cdn_3,cdn_asn,hybrid_ident
30794,30794,http://almezmaah.com/,https://pbs.twimg.com/ad_img/12111589633430364...,image,58740,1,0,Twitter,Twitter,None,None,Edgecast,Twitter
192300,192300,http://reynotch.blog.fc2.com/,https://pbs.twimg.com/ad_img/11937297071008686...,image,189198,1,0,Twitter,Twitter,None,None,Edgecast,Twitter


In [174]:
trackers_by_type = trackers_df['mime_type'].value_counts()

In [175]:
pd.DataFrame({'type_count' : trackers_by_type, 'share' : trackers_by_type/len(trackers_df)}).round(5)

,type_count,share
image,9793122,0.33828
javascript,9650967,0.33337
html,6084385,0.21017
no type,1950048,0.06736
application,693654,0.02396
unidentified,440593,0.01522
text,310073,0.01071
video,26547,0.00092
font,197,0.00001


In [176]:
trackers_by_provider = trackers_df['hybrid_ident'].fillna('- no CDN -').value_counts()

In [177]:
pd.DataFrame({'prov_count' : trackers_by_provider, 'share' : trackers_by_provider/len(trackers_df)}).round(5)

,prov_count,share
Google,15995822,0.55254
- no CDN -,5073329,0.17525
Amazon,2466341,0.08519
Akamai,1170836,0.04044
Facebook,914088,0.03158
Fastly,680578,0.02351
WordPress,598954,0.02069
Twitter,513694,0.01774
Cloudflare,423429,0.01463
Microsoft,323466,0.01117


In [178]:
gc.collect()

0

# HTTP/1.1 vs HTTP/2 ||| TLS versions

In [ ]:
assets_df['protocol'].unique()

In [374]:
assets_df['tls_version'].unique()

array(['', 'TLS 1.2', 'TLS 1.3', 'TLS 1.0', 'TLS 1.1'], dtype=object)

In [375]:
assets_df['protocol'].value_counts()

HTTP/2      164175175
http/1.1    141112092
             17784900
http/1.0       180233
http/0.9          308
Name: protocol, dtype: int64

In [376]:
assets_df['tls_version'].value_counts()

           280768391
TLS 1.2     28380189
TLS 1.3     13641232
TLS 1.0       455884
TLS 1.1         7012
Name: tls_version, dtype: int64

In [398]:
http_providers = pd.crosstab(assets_df['hybrid_ident'].fillna('- no CDN -'),
                             assets_df['protocol'].replace('', '- unspecified -'),
                             margins=True)

In [399]:
http_providers

protocol,- unspecified -,HTTP/2,http/0.9,http/1.0,http/1.1,All
hybrid_ident,,,,,,
- no CDN -,8832227,36990252,282,154054,102208567,148185382
AT&T,11054,13595,0,409,141097,166155
Advanced Hosters CDN,0,0,0,0,44637,44637
Airee,0,62392,0,0,1413,63805
Akamai,1423783,5804986,1,413,5035785,12264968
Amazon CloudFront,4628468,14156851,20,21925,13827948,32635212
"Amazon CloudFront, CDN",0,15,0,0,0,15
"Amazon CloudFront, CDNetworks",0,0,0,0,78,78
"Amazon CloudFront, Cloudflare",0,448305,0,0,14658,462963


In [402]:
http_providers.sort_values(by=['All'], ascending=False).head(12)

protocol,- unspecified -,HTTP/2,http/0.9,http/1.0,http/1.1,All
hybrid_ident,,,,,,
All,17784900,164175175,308,180233,141112092,323252708
- no CDN -,8832227,36990252,282,154054,102208567,148185382
Google,1124012,50740665,0,147,5224063,57088887
Amazon CloudFront,4628468,14156851,20,21925,13827948,32635212
Cloudflare,221401,16347608,0,0,3645609,20214618
Facebook,2152,15105257,0,214,76242,15183865
Akamai,1423783,5804986,1,413,5035785,12264968
Fastly,383828,6892651,0,0,1273676,8550155
Edgecast,286321,1741893,0,0,1853377,3881591


In [410]:
(http_providers['HTTP/2']/164175175).round(5).sort_values(ascending=False).head(15)

hybrid_ident
All                  1.00000
Google               0.30906
- no CDN -           0.22531
Cloudflare           0.09957
Facebook             0.09201
Amazon CloudFront    0.08623
Fastly               0.04198
Akamai               0.03536
Twitter              0.01987
WordPress            0.01873
Edgecast             0.01061
NetDNA               0.00800
Microsoft Azure      0.00703
Highwinds            0.00596
Yahoo                0.00413
Name: HTTP/2, dtype: float64

In [400]:
tls_providers = pd.crosstab(assets_df['hybrid_ident'].fillna('- no CDN -'),
                            assets_df['tls_version'].replace('', '- unspecified -'),
                            margins=True)

In [401]:
tls_providers

tls_version,- unspecified -,TLS 1.0,TLS 1.1,TLS 1.2,TLS 1.3,All
hybrid_ident,,,,,,
- no CDN -,135862917,408358,6671,11542019,365417,148185382
AT&T,142121,1032,5,22970,27,166155
Advanced Hosters CDN,44210,0,0,427,0,44637
Airee,61985,0,0,1820,0,63805
Akamai,10022351,32,0,2230432,12153,12264968
Amazon CloudFront,27219070,37361,82,5372853,5846,32635212
"Amazon CloudFront, CDN",11,0,0,4,0,15
"Amazon CloudFront, CDNetworks",60,0,0,18,0,78
"Amazon CloudFront, Cloudflare",386492,0,0,50121,26350,462963


In [43]:
323252708280768391

42484317

In [42]:
280768391/323252708

0.8685724328100601

In [50]:
280768391 - 71910300

208858091

In [51]:
208858091 / 323252708

0.6461139716113375

In [403]:
tls_providers.sort_values(by=['All'], ascending=False).head(12)

tls_version,- unspecified -,TLS 1.0,TLS 1.1,TLS 1.2,TLS 1.3,All
hybrid_ident,,,,,,
All,280768391,455884,7012,28380189,13641232,323252708
- no CDN -,135862917,408358,6671,11542019,365417,148185382
Google,44588327,217,8,4645035,7855300,57088887
Amazon CloudFront,27219070,37361,82,5372853,5846,32635212
Cloudflare,18671051,8,0,353976,1189583,20214618
Facebook,12615638,0,0,6691,2561536,15183865
Akamai,10022351,32,0,2230432,12153,12264968
Fastly,7467720,4,0,1081376,1055,8550155
Edgecast,3274587,0,0,5668,601336,3881591


In [412]:
# TLS 1.3 share compared to other TLS versions
(tls_providers['TLS 1.3']/(tls_providers['All']-tls_providers['- unspecified -'])
).round(5).sort_values(ascending=False).head(15)

hybrid_ident
All                              1.00000
Google                           0.57585
Facebook                         0.18778
Cloudflare                       0.08720
Edgecast                         0.04408
WordPress                        0.03554
- no CDN -                       0.02679
Twitter                          0.00758
Microsoft Azure                  0.00577
CDN77                            0.00565
jsDelivr                         0.00405
Incapsula                        0.00375
Yahoo                            0.00344
KeyCDN                           0.00269
Amazon CloudFront, Cloudflare    0.00193
Name: TLS 1.3, dtype: float64

In [45]:
non_https = assets_df[(assets_df['tls_version'] == '') & (assets_df['asset_url'].str[:5] != 'https')]

In [46]:
non_https  # 71910300

,index,base_page,asset_url,protocol,tls_version,asset_content_type,mime_type,object_size,is_ad,is_tracker,ip_addr,asset_cdn,cdn_1,cdn_2,cdn_3,asn,holder,cdn_asn,hybrid_ident
0,0,http://www.markreviewsmovies.com/,http://www.markreviewsmovies.com/images/header...,http/1.1,,image/gif,image,12433,0,0,65.254.248.179,,None,None,None,29873,BIZLAND-SD - The Endurance International Group,None,None
1,1,http://www.markreviewsmovies.com/,http://pagead2.googlesyndication.com/pagead/sh...,http/1.1,,text/javascript,javascript,24822,1,0,74.125.142.157,Google,Google,None,None,15169,GOOGLE - Google LLC,Google,Google
2,2,http://www.markreviewsmovies.com/,http://www.markreviewsmovies.com/images/header...,http/1.1,,image/gif,image,321,0,0,65.254.248.179,,None,None,None,29873,BIZLAND-SD - The Endurance International Group,None,None
3,3,http://www.markreviewsmovies.com/,http://www.markreviewsmovies.com/images/header...,http/1.1,,image/gif,image,596,0,0,65.254.248.179,,None,None,None,29873,BIZLAND-SD - The Endurance International Group,None,None
4,4,http://www.markreviewsmovies.com/,http://www.markreviewsmovies.com/images/header...,http/1.1,,image/gif,image,714,0,0,65.254.248.179,,None,None,None,29873,BIZLAND-SD - The Endurance International Group,None,None
5,5,http://www.markreviewsmovies.com/,http://www.markreviewsmovies.com/images/header...,http/1.1,,image/gif,image,422,0,0,65.254.248.179,,None,None,None,29873,BIZLAND-SD - The Endurance International Group,None,None
6,6,http://www.markreviewsmovies.com/,http://pagead2.googlesyndication.com/pagead/js...,http/1.1,,text/javascript,javascript,74263,1,0,74.125.142.157,Google,Google,None,None,15169,GOOGLE - Google LLC,Google,Google
7,7,http://www.markreviewsmovies.com/,http://www.markreviewsmovies.com/images/header...,http/1.1,,image/gif,image,271,0,0,65.254.248.179,,None,None,None,29873,BIZLAND-SD - The Endurance International Group,None,None
8,8,http://www.markreviewsmovies.com/,http://www.markreviewsmovies.com/images/poster...,http/1.1,,image/jpeg,image,35511,0,0,65.254.248.179,,None,None,None,29873,BIZLAND-SD - The Endurance International Group,None,None
9,9,http://www.markreviewsmovies.com/,http://www.markreviewsmovies.com/images/poster...,http/1.1,,image/jpeg,image,40657,0,0,65.254.248.179,,None,None,None,29873,BIZLAND-SD - The Endurance International Group,None,None


In [39]:
tls_providers_bp = pd.crosstab(base_pages['hybrid_ident'].fillna('- no CDN -'),
                               base_pages['tls_version'].replace('', '- unspecified -'),
                               margins=True)

In [40]:
tls_providers_bp.sort_values(by=['All'], ascending=False).head(12)

tls_version,- unspecified -,TLS 1.0,TLS 1.1,TLS 1.2,TLS 1.3,All
hybrid_ident,,,,,,
All,1307738,34958,727,1732895,359049,3435367
- no CDN -,1045661,32674,678,1245736,107259,2432008
Cloudflare,73289,29,0,33842,189393,296553
Amazon CloudFront,64845,1019,19,219839,4295,290017
Google,51644,67,3,77964,12856,142534
Microsoft Azure,11154,231,11,38763,584,50743
Akamai,7998,38,1,40469,255,48761
Fastly,13321,8,0,26336,48,39713
Rackspace,5775,345,8,16382,79,22589


In [414]:
gc.collect()

0